In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from curriculum_learning.models.classifier_model import ClassifierModel
from curriculum_learning import utils
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
MODEL_ARCHITECTURE = {
    'conv_block_filters': [16, 32, 64],
    'conv_block_kernel_sizes': [3, 3, 3],
    'conv_block_strides': [2, 2, 2],
    'conv_block_dropout_rates': [0.2, 0.2, 0.2],
    'dense_block_units': [32],
    'dense_block_dropout_rates': [0.5]
}

N_EPOCHS = 50
N_TRIALS = 30
BATCH_SIZE = 512

loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [30]:
x = []
y = []

ds_1 = tfds.load("stl10", split="train", as_supervised=True, shuffle_files=False)
for x_, y_ in ds_1.as_numpy_iterator():
    x.append(x_)
    y.append(y_)
    
ds_2 = tfds.load("stl10", split="test", as_supervised=True, shuffle_files=False)
for x_, y_ in ds_2.as_numpy_iterator():
    x.append(x_)
    y.append(y_)

x = np.array(x, dtype=np.float32) / 255
y = np.array(y, dtype=np.float32)

2024-07-01 18:45:35.863293: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-07-01 18:45:36.608403: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42, stratify=y)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

n_classes = len(np.unique(y))
train_size = x_train.shape[0]
train_size, len(x_val), len(x_test)

(9100, 1950, 1950)

In [32]:
x_train_sorted = x_train[np.argsort(y_train)]
y_train_sorted = y_train[np.argsort(y_train)]
_, counts = np.unique(y_train_sorted, return_counts=True)

In [48]:
model = ClassifierModel(output_shape=n_classes, **MODEL_ARCHITECTURE)
model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])
model(x_train[0:1])
# model.save_weights("../models/default_model.weights.h5")
model.load_weights("../models/default_model.weights_stl10.h5")
model_weights = model.get_weights()

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '../models/default_model.weights_stl10.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [36]:
samples_values = utils.calculate_values_edges(x_train_sorted, blur=True)

In [37]:
samples_proba = utils.normalize_values_per_group(samples_values, counts)

In [9]:
model_scores = []
verbose = 0
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=5, start_from_epoch=0)

# samples_values = utils.calculate_values_edges(x_train_sorted, counts, blur=True)
# samples_proba = utils.normalize_losses_per_group(samples_values, counts)

ORDER_TYPE = utils.OrderType.FIXED

for _ in tqdm(range(N_TRIALS)):
    model.set_weights(model_weights)

    for i in range(N_EPOCHS):
        n_samples = int(np.tanh(4 * (i + 1) / N_EPOCHS) * train_size)

        samples_values = utils.calculate_values_losses(
            model, x_train_sorted, y_train_sorted, batch_size=BATCH_SIZE
        )
        samples_proba = utils.normalize_values_per_group(samples_values, counts)
        
        samples_ids = utils.chose_samples(n_samples, samples_proba, ORDER_TYPE)

        model.fit(
            x_train_sorted[samples_ids],
            y_train_sorted[samples_ids],
            epochs=1,
            batch_size=BATCH_SIZE,
            shuffle=True,
            verbose=verbose,
        )

    model.fit(
        x_train_sorted, y_train_sorted, validation_data=(x_val, y_val),
        epochs=500, batch_size=BATCH_SIZE, shuffle=True,
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=5, start_from_epoch=10)], 
        verbose=verbose
    )
            
    _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=0)
    model_scores.append(accuracy)
    print(f"Mean: {np.mean(model_scores):.4f}   Median: {np.median(model_scores):.4f}   Last accuracy {accuracy:.4f}")

  0%|          | 0/30 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [16]:
model_scores

[0.8392592668533325,
 0.8106172680854797,
 0.8518518805503845,
 0.8118518590927124,
 0.8627160787582397,
 0.7886419892311096,
 0.8288888931274414,
 0.8254321217536926,
 0.82419753074646,
 0.8402469158172607,
 0.7856789827346802,
 0.8083950877189636,
 0.8318518400192261,
 0.8059259057044983,
 0.8204938173294067,
 0.8283950686454773,
 0.7212345600128174,
 0.760493814945221,
 0.8012345433235168,
 0.7787654399871826,
 0.8548148274421692,
 0.8037037253379822,
 0.8424691557884216,
 0.7701234817504883,
 0.8392592668533325,
 0.7866666913032532,
 0.7990123629570007,
 0.8306173086166382,
 0.8404937982559204,
 0.843950629234314]

In [19]:
model_scores_random = []
verbose = 0

for _ in tqdm(range(N_TRIALS)):
    model.set_weights(model_weights)

    model.fit(
        x_train_sorted, y_train_sorted, validation_data=(x_val, y_val),
        epochs=500, batch_size=BATCH_SIZE, verbose=verbose, shuffle=True, 
        callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', restore_best_weights=True, patience=5, start_from_epoch=35)]
    )

    _, accuracy = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE, verbose=verbose)
    model_scores_random.append(accuracy)
    print(f"Mean: {np.mean(model_scores):.4f}   Median: {np.median(model_scores):.4f}   Last accuracy {accuracy:.4f}")